# Tugas 1
# Prediksi Kemunculan Kata dan Kalimat Menggunakan Model Bigram

Dwiferdio Seagal Putra (1301154323)

Muhammad Adnan Rizqullah (1301154228)

Hafid Junur (1103130285)

# Import Modul

In [53]:
from nltk.tokenize import word_tokenize
import string
from collections import Counter
import re
from itertools import chain
from math import log10, exp
from functools import reduce

# Pendefinisian Fungsi Program

In [2]:
# Fungsi untuk membaca semua file pada folder txt 
# menjadi satu list kumpulan berita
# Input : null
# Output : List text berita

def get_texts():
    f_path = "article/detikcom/"
    f_names = []
    for i in range(0,153):
        f_names.append("detik_" + str(i) + ".txt")

    texts = []
    for f_name in f_names:
        with open(f_path + f_name) as f:
            new_text = f.read()
            texts.append(new_text)
    return texts

In [3]:
# Fungsi mencari dan mendapatkan substring editor notes
# Menerima text yang berisikan editor notes
# Mengeluarkan editor notes

def get_editor_notes(editor_txt):
    match = re.search(r'\([a-z]+/[a-z]+\).*', editor_txt)
    if match:
        editor_notes = match.group()
    return editor_notes

In [4]:
# Fungsi menghilangkan editor notes dari sebuah text
# Input : Text kotor berisikan editor notes
# Output : Text bersih/bebas dari editor notes

def get_editor_free_text(editor_notes, dirty_txt):
    idx = dirty_txt.find(editor_notes)
    editor_free = dirty_txt[:idx]
    return editor_free

In [5]:
# Prosedur untuk menghilangkan setiap editor notes dari
# semua berita pada sebuah list
# Input : List kotor berisikan berita kotor (editor notes)
# dan List kosong penampung hasil
# Output : List bersih berisikan berita bersih dari editor
# notes

def remove_editor_notes(editor_free_texts, editor_texts):
    for index, text in enumerate(editor_texts):
        try:
            editor_notes = get_editor_notes(text)
        except UnboundLocalError:
            print("Editor notes not found!")
        else:
            editor_free_txt = get_editor_free_text(editor_notes, text)
            editor_free_texts.append(editor_free_txt)

In [6]:
# Fungsi mengubah list text menjadi list token
# Input : List text berita
# Output : List token berita

def get_tokens(texts):
    list_of_tokens = []
    for text in texts:
        tokens = word_tokenize(text)
        list_of_tokens.append(tokens)
    return list_of_tokens

In [7]:
# Fungsi menambah <s> dan <\s> pada setiap token berita 
# dari list token berita pada setiap kalimat
# Input : List token berita
# Output : List token berita dengan <s> dan <\s>

def get_sentence_seperator_list(list_of_tokens):
    list_of_sentence_tokens = []
    for tokens in list_of_tokens:
        if (tokens[-1] != ".") and (tokens[-1] != "?") and (tokens[-1] != "!"):
            tokens.append(".")
    for tokens in list_of_tokens:
        tokens.insert(0, "<s>")
        for index, token in enumerate(tokens):
            if (token == ".") or (token == "?") or (token == "!"):
                tokens[index] = "<\s>"
                tokens.insert(index + 1, "<s>")
        del tokens[-1]
        list_of_sentence_tokens.append(tokens)
    return list_of_sentence_tokens

In [8]:
# Memperbaiki bug dari tokenisasi NLTK berupa
# angka dan persentasenya yang terpisah
# Input : List token berita
# Output : list token berita yang diperbaiki

def fix_percentage_problem(list_of_sTokens):
    list_of_pTokens = []
    for tokens in list_of_sTokens:
        for index, token in enumerate(tokens):
            if token == "%":
                old_word = tokens[index - 1]
                new_word = old_word +"%"
                tokens[index - 1] = new_word
        list_of_pTokens.append(tokens)
    return list_of_pTokens

In [9]:
# Memperbaiki bug dari kesalahan penulisan
# detik.com berupa dua kalimat yang tergabung
# dengan "."
# Input : List token berita
# Output : list token berita yang diperbaiki

def fix_dot_problem(list_of_pTokens):
    list_of_dTokens = []
    for tokens in list_of_pTokens:
        for index, token in enumerate(tokens):
            if "." in token:
                dot_index = token.find(".")
                before_dot = token[:dot_index]
                after_dot = token[dot_index + 1:]
                del tokens[index]
                tokens.insert(index, before_dot)
                tokens.insert(index + 1, after_dot)
                tokens.insert(index + 1, "<\s>")
                tokens.insert(index + 2, "<s>")
        list_of_dTokens.append(tokens)
    return list_of_dTokens

In [10]:
# Menghilangkan token tanda baca dari setiap berita
# pada list token berita
# Input : List token berita
# Output : list token berita tanpa tanda baca

def remove_punctuation(list_of_dTokens):
    list_of_puncTokens = []
    puncts = string.punctuation
    puncts += "''``"
    for tokens in list_of_dTokens:
        clean_tokens = [token for token in tokens if token not in puncts]
        list_of_puncTokens.append(clean_tokens)
    return list_of_puncTokens

In [11]:
def make_lowercase(list_of_puncTokens):
    list_of_cleanT = []
    for tokens in list_of_puncTokens:
        tmp = []
        for token in tokens:
            tmp.append(token.lower())
        list_of_cleanT.append(tmp)
    return list_of_cleanT

In [12]:
# Fungsi semua preprocessing, menerima tokens hasil tokenisasi NLTK. 
# Mengeluarkan token bersih, hasil preprocessing

def get_clean_tokens(texts):
    # 1. Menghilangkan editor notes dari semua berita
    editor_free_texts = []
    remove_editor_notes(editor_free_texts, texts)
    
    # 2. Tokenisasi semua teks berita
    list_of_tokens = get_tokens(editor_free_texts)
    
    # 3. Menambahkan <s> dan <\s> pada setiap kalimat
    list_of_sTokens = get_sentence_seperator_list(list_of_tokens) 

    # 4. Menyatukan dua token terpisah berupa angka dan persentase yang
    # Terpisah setelah tokenisasi NLTK. Seperti : ['25', %] -> ['25%']
    list_of_pTokens = fix_percentage_problem(list_of_sTokens)
            
    # 5. Memisahkan dua kata yang menjadi satu karena salah tulis pada
    # Awal dan akhir kalimat. Lalu menambahkan <s> dan <\s> pada
    # Tempatnya. Seperti : ["DNS.Parahnya"] -> ["DNS", "<\s>","<s>", "Parahnya"]
    list_of_dTokens = fix_dot_problem(list_of_pTokens)
    
    # 6. Menghilangkan tanda baca
    list_of_puncTokens = remove_punctuation(list_of_dTokens)
    
    # 7. Mengubah semua kata menjadi lowercase
    list_of_cleanT = make_lowercase(list_of_puncTokens)
    return list_of_cleanT

In [13]:
# Fungsi membuat bigram dan countnya. Menerima token bersih berita
# Mengeluarkan bigram matrix

def get_bigram_matrix(clean_tokens):
    bigrams = []
    for i in range(len(clean_tokens) - 1 ):
        bigrams.append((clean_tokens[i], clean_tokens[i + 1]))
    bi_matrix = Counter(bigrams)
    return(bi_matrix)

In [21]:
def get_bigram(clean_tokens):
    bigrams = []
    for i in range(len(clean_tokens)-1):
        bigrams.append((clean_tokens[i], clean_tokens[i+1]))
    return bigrams

In [14]:
# Fungsi menghitung probabilitas kemunculan sebuah kalimat
# Input : kalimat
# Output : probilitas dalam decimal
# mengeluarkan nilai 0 jika terdapat kata pada kalimat
# yang tak dikenal. Dengan Implementasi Laplace

def get_sentence_probability(sentence):
    test = word_tokenize(sentence.lower())
    punct = string.punctuation
    test_clean = [token for token in test if token not in punct]
    test_bigram = [(test_clean[i], test_clean[i+1]) for i in range(len(test_clean) - 1)]
    start = "<s>"
    end = "<\s>"
    test_bigram.insert(0, (start, test_bigram[0][0]))
    test_bigram.append((test_bigram[-1][-1], end))

    probability = 0
    log_probability = 0
    # Implementasi prediksi kumunculan kalimat + Laplace
    for tup in test_bigram:
        try:
            pembagi = count_token[tup[0]] + len(count_token)
        except KeyError:
            pembagi = len(count_token)
            pembilang = 1
            log_probability = log10(pembilang/pembagi)
        else:
            try:
                pembilang = count_bigrams[tup] + 1
            except KeyError:
                pembilang = 1
                log_probability = log10(pembilang/pembagi)
            else:
                log_probability = log10(pembilang/pembagi)
    probability = exp(log_probability)
    return probability

In [48]:
def get_probability(word, prev_word, length_corpus, list_token, list_bigrams):
    '''
    Fungsi untuk menghitung probabilitas dari suatu kalimat
    '''
    count_token = dict(Counter(list_token))
    count_bigrams = dict(Counter(list_bigrams))
    
    try:
        return (count_bigrams[word] + 1) / (count_token[prev_word] + length_corpus)    
    except KeyError:
        try:
            return 1 / (count_token[prev_word] + length_corpus)
        except Exception:
            return 0

In [15]:
# Fungsi memprediksi kata selanjutnya
# Input : kalimat
# Output : kata selanjutnya

def get_next_word(sentence): 
    punct = string.punctuation.replace("%", "")
    clean_token = [token for token in word_tokenize(sentence.lower()) if token not in punct]
    last_word = clean_token[-1]

    max_value = -1
    word_prediction = ""
    idx = 0
    for tup, count in count_bigrams.items():
        if last_word == tup[0]:
            if tup[1] != "<\s>":
                if max_value < count:
                    max_value = count
                    word_prediction = tup[1]
    if word_prediction == "":
        return "Error, kata terakhir tak dikenal"
    else:
        return word_prediction

# Pembangunan Model Bigram

In [16]:
# Preprocessing
# 1. Menghilangkan catatan editor dari setiap berita
# 2. Tokensiasi semua berita
# 3. Menambah <s> dan <\s>
# 4. Mengatasi bug karena library NLTK
# 5. Mengatasi bug karena kesalahan penulisan detik.com
# 6. Menghilangkan tanda baca
# 7. Mengubah semua token menjadi lowercase

texts = get_texts()
list_of_cleanT = get_clean_tokens(texts)

In [17]:
# Konversi 2-D list ke 1-D list dan membuat Bigram Matrix

list_final_tokens = list(chain.from_iterable(list_of_cleanT))
bigram_matrix = get_bigram_matrix(list_final_tokens)

In [18]:
# Mendapatkankan count setiap token dan tuple

count_token = dict(Counter(list_final_tokens))
count_bigrams = dict(bigram_matrix)

### Menghitung perplexity

In [64]:
# Fungsi untuk menghitung perplexity dari suatu kalimat
# Input : Kalimat
# Ouput : Perplexity dari kalimat tersebut
def get_perplexity(sentences):
    n_root = lambda number, root: number ** (1/root)
    words = word_tokenize(sentences)
    bigram = get_bigram(words)
    length_corpus = len(count_token)

    probability = [get_probability(bigram[i], bigram[i][0], length_corpus, list_final_tokens, bigram_matrix) for i in range(len(bigram))]

    non_zero_probability = list(filter(lambda x: x != 0, probability))
    inv_probability = []
    for i in range(len(non_zero_probability)):
        inv_probability.append(1/non_zero_probability[i])
        temp_perplexity = reduce(lambda x, y: x*y, inv_probability)
        perplexity = n_root(temp_perplexity, len(words))
    
    return perplexity

In [59]:
# Menghitung perplexity untuk kalimat pada pengujian probabilitas kemunculan sebuah kalimat
sentence = "Setelah itu barulah pihak bank mempolisikan Qisheng"
perplexity = get_perplexity(sentence)
print(perplexity)

325.46056398758174


In [60]:
# Menghitung perplexity untuk kalimat pada pengujian probabilitas kemunculan sebuah kalimat
sentence = "WhatsApp akan mengendus adanya tautan yang dianggap mencurigakan pada chat"
perplexity = get_perplexity(sentence)
print(perplexity)

627.7278990120345


In [61]:
# Menghitung perplexity untuk kalimat pada pengujian probabilitas kemunculan sebuah kalimat
sentence = "Keberadaan sebuah alat untuk meretas iPhone secara remote terungkap"
perplexity = get_perplexity(sentence)
print(perplexity)

218.34306404099456


In [62]:
# Menghitung perplexity untuk kalimat pada pengujian probabilitas kemunculan sebuah kalimat
sentence = "Pemilik rekening diduga menyuntikkan transaksi"
perplexity = get_perplexity(sentence)
print(perplexity)

177.47042945362318


In [63]:
# Menghitung perplexity untuk kalimat pada pengujian probabilitas kemunculan sebuah kalimat
sentence = "Polisi menghentikan kasus pasar saham"
perplexity = get_perplexity(sentence)
print(perplexity)

154.65439791566726


# Prediksi Model

### 1. Probabilitas Kemunculan Sebuah Kalimat

In [23]:
'''
Menghitung probabilitas kemunculan sebuah kalimat
Kasus Uji 1

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir sesuai korpus. Mau diperiksa kemampuan dalam memprediksi kata yang dikenal persis

'''
sentence = "Setelah itu barulah pihak bank mempolisikan Qisheng"
probabilitas = get_sentence_probability(sentence)
print(probabilitas)

0.03431912917479203


In [24]:
'''
Menghitung probabilitas kemunculan sebuah kalimat
Kasus Uji 2

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir sesuai korpus. Mau diperiksa kemampuan dalam memprediksi kata yang dikenal persis

'''
sentence = "WhatsApp akan mengendus adanya tautan yang dianggap mencurigakan pada chat"
probabilitas = get_sentence_probability(sentence)
print(probabilitas)

0.028747995206027978


In [25]:
'''
Menghitung probabilitas kemunculan sebuah kalimat
Kasus Uji 3

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir sesuai korpus. Mau diperiksa kemampuan dalam memprediksi kata yang dikenal persis

'''
sentence = "Keberadaan sebuah alat untuk meretas iPhone secara remote terungkap"
probabilitas = get_sentence_probability(sentence)
print(probabilitas)

0.028785090479781318


In [26]:
'''
Menghitung probabilitas kemunculan sebuah kalimat
Kasus Uji 4

Alasan pemilihan : 
Kalimat dipilih dari kata yang ada di korpus dan disusun sesuai dengan kaidah berbahasa

'''
sentence = "Pemilik rekening diduga menyuntikkan transaksi"
probabilitas = get_sentence_probability(sentence)
print(probabilitas)

0.03431280307578489


In [27]:
'''
Menghitung probabilitas kemunculan sebuah kalimat
Kasus Uji 5

Alasan pemilihan : 
Kalimat dipilih dari kata yang ada di korpus dan disusun sesuai dengan kaidah berbahasa

'''
sentence = "Polisi menghentikan kasus pasar saham"
probabilitas = get_sentence_probability(sentence)
print(probabilitas)

0.028772707828094803


### 2. Prediksi Kemunculan Kata Berikutnya

In [20]:
'''
Prediksi kemunculan suatu kata berikutnya
Kasus Uji 1

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir
sesuai korpus. Juga kata mempolisikan hanya diikuti oleh "qisheng".
Mau diperiksa kemampuan dalam memprediksi kalimat yang sama persis
dengan korpus. Output harus "qisheng"

'''
sentence = "Setelah itu barulah pihak bank mempolisikan"
word_prediction = get_next_word(sentence)
print(word_prediction)

qisheng


In [28]:
'''
Prediksi kemunculan suatu kata berikutnya
Kasus Uji 2

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir
sesuai korpus. Juga kata mempolisikan hanya diikuti oleh "qisheng".
Mau diperiksa kemampuan dalam memprediksi kalimat yang sama persis
dengan korpus. Output harus "qisheng"

'''

sentence = "WhatsApp akan mengendus adanya"
word_prediction = get_next_word(sentence)
print(word_prediction)

celah


In [29]:
'''
Prediksi kemunculan suatu kata berikutnya
Kasus Uji 3

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir
sesuai korpus. Juga kata mempolisikan hanya diikuti oleh "qisheng".
Mau diperiksa kemampuan dalam memprediksi kalimat yang sama persis
dengan korpus. Output harus "qisheng"

'''

sentence = "Keberadaan sebuah alat untuk meretas"
word_prediction = get_next_word(sentence)
print(word_prediction)

iphone


In [30]:
'''
Prediksi kemunculan suatu kata berikutnya
Kasus Uji 4

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir
sesuai korpus. Juga kata mempolisikan hanya diikuti oleh "qisheng".
Mau diperiksa kemampuan dalam memprediksi kalimat yang sama persis
dengan korpus. Output harus "qisheng"

'''

sentence = "Kloningan yang menyamar sebagai"
word_prediction = get_next_word(sentence)
print(word_prediction)

informasi


In [31]:
'''
Prediksi kemunculan suatu kata berikutnya
Kasus Uji 5

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir
sesuai korpus. Juga kata mempolisikan hanya diikuti oleh "qisheng".
Mau diperiksa kemampuan dalam memprediksi kalimat yang sama persis
dengan korpus. Output harus "qisheng"

'''

sentence = "Aplikasi dan game kloningan"
word_prediction = get_next_word(sentence)
print(word_prediction)

yang


In [32]:
'''
Prediksi kemunculan suatu kata berikutnya
Kasus Uji 6

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir
sesuai korpus. Juga kata mempolisikan hanya diikuti oleh "qisheng".
Mau diperiksa kemampuan dalam memprediksi kalimat yang sama persis
dengan korpus. Output harus "qisheng"

'''

sentence = "Pemilik rekening diduga menyuntikkan"
word_prediction = get_next_word(sentence)
print(word_prediction)

sebuah


In [33]:
'''
Prediksi kemunculan suatu kata berikutnya
Kasus Uji 7

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir
sesuai korpus. Juga kata mempolisikan hanya diikuti oleh "qisheng".
Mau diperiksa kemampuan dalam memprediksi kalimat yang sama persis
dengan korpus. Output harus "qisheng"

'''

sentence = "Polisi menghentikan kasus"
word_prediction = get_next_word(sentence)
print(word_prediction)

ini


In [37]:
'''
Prediksi kemunculan suatu kata berikutnya
Kasus Uji 8

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir
sesuai korpus. Juga kata mempolisikan hanya diikuti oleh "qisheng".
Mau diperiksa kemampuan dalam memprediksi kalimat yang sama persis
dengan korpus. Output harus "qisheng"

'''

sentence = "Pembobol digrebek polisi"
word_prediction = get_next_word(sentence)
print(word_prediction)

saat


In [38]:
'''
Prediksi kemunculan suatu kata berikutnya
Kasus Uji 9

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir
sesuai korpus. Juga kata mempolisikan hanya diikuti oleh "qisheng".
Mau diperiksa kemampuan dalam memprediksi kalimat yang sama persis
dengan korpus. Output harus "qisheng"

'''
sentence = "Beberapa penipuan menjalankan"
word_prediction = get_next_word(sentence)
print(word_prediction)

sistem


In [42]:
'''
Prediksi kemunculan suatu kata berikutnya
Kasus Uji 10

Alasan pemilihan : 
Ini kalimat persis dari awal hingga akhir
sesuai korpus. Juga kata mempolisikan hanya diikuti oleh "qisheng".
Mau diperiksa kemampuan dalam memprediksi kalimat yang sama persis
dengan korpus. Output harus "qisheng"

'''

sentence = "Konten-konten pada imessage tersimpan"
word_prediction = get_next_word(sentence)
print(word_prediction)

di
